In [2]:
import os
import re
import ast
import chardet
import getpass

from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage
from langchain_community.document_loaders import TextLoader

from prompt import MERGE_CODE

os.environ["OPENAI_API_KEY"] = "sk-kTtVqDFBwOc4coa2HKva2rlSmHVufJ07FgL6uilOScGMT6Uc"
os.environ["OPENAI_BASE_URL"] = "https://api.chatanywhere.tech"

model = ChatOpenAI(model="gpt-4o-mini")


def code_merge(old_code: str, code_error, code_analysis: str):
    merge_model = ChatOpenAI(model="gpt-4o-mini", temperature=0)
    merge_input = (
        MERGE_CODE
        + "原代码为："
        + old_code
        + "代码的错误为："
        + code_error
        + "错误分析为："
        + code_analysis
    )

    return merge_model.invoke([HumanMessage(content=merge_input)]).content


# 返回字典对象
def file_loader(file_name: str):
    loader = TextLoader(file_name, encoding="utf-8")
    documents = loader.load()
    code = documents[0].page_content
    return {"file_name": file_name, "content": code}


def detect_file_encoding(file_path, sample_size=10000):
    with open(file_path, "rb") as f:
        raw_data = f.read(sample_size)  # 读取前10KB用于检测
    result = chardet.detect(raw_data)
    return result["encoding"], result["confidence"]


def add_line_numbers_to_string(file):
    lines = file.split("\n")

    modified_content = ""
    for idx, line in enumerate(lines, start=1):
        # 添加行号信息并追加到modified_content字符串变量中
        modified_line = f"{line.rstrip()}  # 第{idx}行\n"
        modified_content += modified_line
    return modified_content  # 返回包含行号的新字符串内容


def delete_line(text, start, end):
    # 将文本按行分割成列表
    lines = text.splitlines()

    # 删除指定行（注意索引从0开始)
    delete_line_nums = end - start + 1
    for i in range(delete_line_nums):
        lines.pop(start - 1)

    # 重新组合成单个字符串
    modified_text = "\n".join(lines)

    return modified_text


def add_line(text, start, insert_content):
    # 将文本按行分割成列表
    lines = text.splitlines()

    # 插入新内容到指定行前
    if 1 <= start <= len(lines) + 1:
        lines.insert(start - 1, insert_content)

    # 重新组合成单个字符串
    modified_text = "\n".join(lines)

    return modified_text


def code_extract(llm_code: str):
    pattern = r"(.*?)```"
    match = re.search(pattern, llm_code, re.DOTALL)
    if match:
        before_code_block = match.group(1).strip()

        # extract code
        code_pattern = r"```(?:\w+)?(.*?)(?:```|$)"
        code_match = re.search(code_pattern, llm_code, re.DOTALL)
        if code_match:
            code_block = code_match.group(1).strip()
            print(code_block)
            return code_block
    else:
        return llm_code

### 得到repair code snippet

In [3]:
# problem = "这段代码统计学生平均分数逻辑错了。"+"帮我修改，只需要输出需要改进的代码结果"
problem = "这段代码缺少一个统计单个学生平均成绩的功能，帮我添加。这段代码的add_grade方法可读性差，帮我修改。这段代码有一个重复的功能，帮我删除。"
problem=problem+"根据问题和原代码，输出应该怎么修改原代码，以及修改后的代码（只需要修改的部分，不要生成全部代码）"

In [4]:
code = file_loader("./buggy.py")["content"]
msg = HumanMessage(content=code + problem)
resp = model.invoke([msg])
print(resp.content)

根据你的要求，我们需要添加一个统计单个学生平均成绩的功能，改进 `add_grade` 方法的可读性，删除重复的 `show_all_students` 函数。以下是需要修改的部分：

### 修改部分

1. 添加 `calculate_student_average` 方法来计算单个学生的平均成绩。
2. 修改 `add_grade` 方法，以提高可读性。
3. 删除重复的 `show_all_students` 函数。

### 修改后的代码

```python
def calculate_student_average(student_name):
    scores = get_student_grades(student_name)
    if not scores:
        return 0.0
    return sum(scores) / len(scores)

def add_grade(student_name, score):
    student_info = next((student for student in students if student["name"] == student_name), None)
    if student_info:
        student_id = student_info["id"]
        for g in grades:
            if g["student_id"] == student_id:
                g["scores"].append(score)
                break
    else:
        print(f"找不到学生 {student_name}")

# 删除重复的 show_all_students 函数
```

### 修改说明

1. **新增 `calculate_student_average` 方法**：该方法接受学生姓名作为参数，调用 `get_student_grades` 来获得成绩并计算平均值。
   
2. **修改 `add_grade` 方法**：使用 `next()` 函数和生成器表达式来提高查找学生的可读性。

3. **删除第二个 `s

In [5]:
print(code_merge(code, problem, resp.content))

```python
students = []
grades = []


def add_student(name, age):
    student_id = len(students) + 1
    students.append({"id": student_id, "name": name, "age": age})
    grades.append({"student_id": student_id, "scores": []})


def add_grade(student_name, score):
    student_info = next((student for student in students if student["name"] == student_name), None)
    if student_info:
        student_id = student_info["id"]
        for g in grades:
            if g["student_id"] == student_id:
                g["scores"].append(score)
                break
    else:
        print(f"找不到学生 {student_name}")


def show_all_students():
    for student in students:
        print(f"ID: {student['id']}, 姓名: {student['name']}, 年龄: {student['age']}")


def get_student_grades(student_name):
    for student in students:
        if student["name"] == student_name:
            student_id = student["id"]
            break
    else:
        print(f"找不到学生 {student_name}")
        return []

    for g in 

In [4]:
print(add_line_numbers_to_string(code))

students = []  # 第1行
grades = []  # 第2行
  # 第3行
  # 第4行
def add_student(name, age):  # 第5行
    student_id = len(students) + 1  # 第6行
    students.append({"id": student_id, "name": name, "age": age})  # 第7行
    grades.append({"student_id": student_id, "scores": []})  # 第8行
  # 第9行
  # 第10行
def add_grade(student_name, score):  # 第11行
    for student in students:  # 第12行
        if student["name"] == student_name:  # 第13行
            student_id = student["id"]  # 第14行
            break  # 第15行
    else:  # 第16行
        print(f"找不到学生 {student_name}")  # 第17行
        return  # 第18行
  # 第19行
    for g in grades:  # 第20行
        if g["student_id"] == student_id:  # 第21行
            g["scores"].append(score)  # 第22行
            break  # 第23行
  # 第24行
  # 第25行
def show_all_students():  # 第26行
    for student in students:  # 第27行
        print(f"ID: {student['id']}, 姓名: {student['name']}, 年龄: {student['age']}")  # 第28行
  # 第29行
  # 第30行
def get_student_grades(student_name):  # 第31行
    for stude

In [7]:
code_line_fix_format = """
你是一名程序员，你得到了如下内容：
（1）一份有错误的代码，并且每一行代码都标记了行号。
（2）错误描述
（3）对这个错误的修补方案。
---
你要完成以下任务：
将自然语言形式的修补方案按照以下规则转为字典形式。
要求如下：
修补方案可以转换为3种基本操作：删除代码，增加代码，更新代码。

3种方案都需要首先确定一个在原代码里的起始行号 target_line。

对于删除，按照以下格式给出：{flag:del,start:target_line,end:end_line},del表示删除操作，target_line和end_line对应被删除代码的起始行号和结束行号。

对于增加，按照以下格式给出：{flag:add,start:target_line,code_to_add:```python(or other language)
```}, add表示增加操作，target_line代表在新的代码的位置。原代码中target_line行以及之后的代码将被置于新的代码之后。因此请仔细确认行号。code_to_add的代码以一行的形式给出（即使用\n来换行）

对于更新，按照以下格式给出：{flag:update,start:target_line,end:end_line,
code_to_add:```python(or other language)
```}, update表示更新操作，target_line和end_line对应被删除代码的起始行和结束行号（这些代码将被舍弃，因此请仔细确认，不要误删有用的代码），新的代码code_to_add将从target_line行开始替换为新的代码。代码以一行的形式给出（即使用\n来换行）

target_line和end_line始终基于原代码给出的行号，不要使用表达式，只需要原始数字。

可能需要修改多处代码，因此多个操作的起始行号需要严格的从小到大。

注意：每个{}代表一处独立的代码修改，因此只能包含一个起始行号。

每个操作左右两边都被花括号包裹。
你可以输出多个被{}包裹的修改操作，每个操作都用|来分割，如：{操作}|{操作}|{操作}
你需要输出推理过程，但你最终格式化的结果必须用[OUTPUT]作为开始符号，例如：
[OUTPUT]{操作}（如果有多个，就是[OUTPUT]{操作}|{操作}|{操作}。实际输出中，将操作替换为上述提到的格式化的3种操作。

---
step by step的来分析问题，并给出结果
"""

### line提取

In [9]:
print(code_line_fix_format)


你是一名程序员，你得到了如下内容：
（1）一份有错误的代码，并且每一行代码都标记了行号。
（2）错误描述
（3）对这个错误的修补方案。
---
你要完成以下任务：
将自然语言形式的修补方案按照以下规则转为字典形式。
要求如下：
修补方案可以转换为3种基本操作：删除代码，增加代码，更新代码。

3种方案都需要首先确定一个在原代码里的起始行号 target_line。

对于删除，按照以下格式给出：{flag:del,start:target_line,end:end_line},del表示删除操作，target_line和end_line对应被删除代码的起始行号和结束行号。

对于增加，按照以下格式给出：{flag:add,start:target_line,code_to_add:```python(or other language)
```}, add表示增加操作，target_line代表在新的代码的位置。原代码中target_line行以及之后的代码将被置于新的代码之后。因此请仔细确认行号。code_to_add的代码以一行的形式给出（即使用
来换行）

对于更新，按照以下格式给出：{flag:update,start:target_line,end:end_line,
code_to_add:```python(or other language)
```}, update表示更新操作，target_line和end_line对应被删除代码的起始行和结束行号（这些代码将被舍弃，因此请仔细确认，不要误删有用的代码），新的代码code_to_add将从target_line行开始替换为新的代码。代码以一行的形式给出（即使用
来换行）

target_line和end_line始终基于原代码给出的行号，不要使用表达式，只需要原始数字。

可能需要修改多处代码，因此多个操作的起始行号需要严格的从小到大。

注意：每个{}代表一处独立的代码修改，因此只能包含一个起始行号。

每个操作左右两边都被花括号包裹。
你可以输出多个被{}包裹的修改操作，每个操作都用|来分割，如：{操作}|{操作}|{操作}
你需要输出推理过程，但你最终格式化的结果必须用[OUTPUT]作为开始符号，例如：
[OUTPUT]{操作}（如果有多个，就是[OUTPUT]{操作}|{操作}|{操作

In [16]:
msg2 = HumanMessage(
    content=code_line_fix_format
    + "\n代码为："
    + add_line_numbers_to_string(code)
    + "\n问题为"
    + problem
    + "    \n解决方案为"
    + resp.content
)
resp_2 = model.invoke([msg2])

print(resp_2.content)

为了实现上述问题的修复，我将根据补救方案逐步转换为字典形式，并将其格式化为所需的输出。

### 1. 添加单个学生平均成绩的功能
我们需要新增一个函数 `calculate_student_average(student_name)`，应该在第45行插入该函数，因此新增功能的代码如下：
```python
def calculate_student_average(student_name):
    scores = get_student_grades(student_name)
    if not scores:
        return 0.0
    return sum(scores) / len(scores)
```

对应的字典形式是增加操作，起始行号为45。
因此，该操作为：
```python
{flag:add,start:45,code_to_add:```python
def calculate_student_average(student_name):
    scores = get_student_grades(student_name)
    if not scores:
        return 0.0
    return sum(scores) / len(scores)
```}`
```

### 2. 改善 `add_grade` 方法的可读性
我们需要更新 `add_grade` 方法，使其使用 `students_dict` 字典来存储和查找学生信息。`add_student` 函数也需要更新，以便初始化 `students_dict`。相应的修改分为两个部分，具体需要更新的部分如下：

对于 `add_student` 方法，更新如下（以第5行至第8行的更新操作）：
```python
students_dict = {}

def add_student(name, age):
    student_id = len(students) + 1
    student_info = {"id": student_id, "name": name, "age": age}
    students.append(student_info)
    students_dict[name] = st

In [15]:
print(msg2.content)


你是一名程序员，你得到了如下内容：
（1）一份有错误的代码，并且每一行代码都标记了行号。
（2）错误描述
（3）对这个错误的修补方案。
---
你要完成以下任务：
将自然语言形式的修补方案按照以下规则转为字典形式。
要求如下：
修补方案可以转换为3种基本操作：删除代码，增加代码，更新代码。

3种方案都需要首先确定一个在原代码里的起始行号 target_line。

对于删除，按照以下格式给出：{flag:del,start:target_line,end:end_line},del表示删除操作，target_line和end_line对应被删除代码的起始行号和结束行号。

对于增加，按照以下格式给出：{flag:add,start:target_line,code_to_add:```python(or other language)
```}, add表示增加操作，target_line代表在新的代码的位置。原代码中target_line行以及之后的代码将被置于新的代码之后。因此请仔细确认行号。code_to_add的代码以一行的形式给出（即使用
来换行）

对于更新，按照以下格式给出：{flag:update,start:target_line,end:end_line,
code_to_add:```python(or other language)
```}, update表示更新操作，target_line和end_line对应被删除代码的起始行和结束行号（这些代码将被舍弃，因此请仔细确认，不要误删有用的代码），新的代码code_to_add将从target_line行开始替换为新的代码。代码以一行的形式给出（即使用
来换行）

target_line和end_line始终基于原代码给出的行号，不要使用表达式，只需要原始数字。

可能需要修改多处代码，因此多个操作的起始行号需要严格的从小到大。

注意：每个{}代表一处独立的代码修改，因此只能包含一个起始行号。

每个操作左右两边都被花括号包裹。
你可以输出多个被{}包裹的修改操作，每个操作都用|来分割，如：{操作}|{操作}|{操作}
你需要输出推理过程，但你最终格式化的结果必须用[OUTPUT]作为开始符号，例如：
[OUTPUT]{操作}（如果有多个，就是[OUTPUT]{操作}|{操作}|{操作

In [11]:
print(resp_2.content)

根据你的要求，我们将针对指定的错误描述，分析并输出相应的修补方案，按照你所要求的格式进行组织。我们有三个主要的修改：添加新功能、更新现有代码的可读性、删除重复功能。接下来逐步进行分析：

### 1. 添加单个学生平均成绩的功能
我们需要添加一个新的函数 `calculate_student_average(student_name)` 来计算特定学生的平均分。

#### 修补方案：
- 增加操作，新的代码添加的位置在现有代码的结束处，即第80行之后。
```python
def calculate_student_average(student_name):
    scores = get_student_grades(student_name)
    if not scores:
        return 0.0
    return sum(scores) / len(scores)
```

### 2. `add_grade` 方法的可读性改善
我们将改善 `add_grade` 方法的可读性，通过引入一个字典来存储学生信息，以便于更快速和清晰地查找学生。

#### 修补方案：
- 更新操作，更新内容为`add_grade`的内容，涉及从第11行开始的代码（包括换行）。
```python
students_dict = {}
def add_student(name, age):
    student_id = len(students) + 1
    student_info = {"id": student_id, "name": name, "age": age}
    students.append(student_info)
    students_dict[name] = student_info
    grades.append({"student_id": student_id, "scores": []})

def add_grade(student_name, score):
    student_info = students_dict.get(student_name)
    if not student_info:
        print(f"找不到学生 {student_name}")
    

In [14]:
x = [msg2, AIMessage(content=resp_2.content),HumanMessage(content="为什么是11和17？")]
model.invoke(x)

AIMessage(content='感谢你指正。让我来明确一下，给出的代码片段中，`add_grade` 函数的起始行号应该是第11行，并且这个函数的内容包含到第24行，因此更新操作应该涵盖从第11行到第24行。我们再一次检查这个部分，并修正行号。\n\n### 更正分析：\n\n1. **更新 `add_grade` 方法**：\n    - `add_grade` 方法从第11行开始，到第24行结束。所以我们需要更新的区间是从第11行到第24行。\n\n   下面是 `add_grade` 方法的原始行号：\n   ```python\n   def add_grade(student_name, score):  # 第11行\n       for student in students:  # 第12行\n           if student["name"] == student_name:  # 第13行\n               student_id = student["id"]  # 第14行\n               break  # 第15行\n       else:  # 第16行\n           print(f"找不到学生 {student_name}")  # 第17行\n           return  # 第18行\n       ... # 第19行到第24行\n   ```\n\n### 综合修补方案：\n因此，更新操作的行号范围应为11到24行，下面是修补方案的更新：\n\n[OUTPUT]\n{flag:add,start:80,code_to_add:```python\ndef calculate_student_average(student_name):\n    scores = get_student_grades(student_name)\n    if not scores:\n        return 0.0\n    return sum(scores) / len(scores)\n```}|{flag:update,start:11,end:24,\ncode_to_add:```python\nstudents_dict = {}\ndef add_studen

In [27]:
# 每次处理一个代码操作对象
def code_process(
    code: str,
    fix_dict_list: list,
):
    """
    fix元素为字典，如：{"flag":"update","start":53,"end":62,"code_to_add":"```python\ndef calculate_student_average(student_name): \n    scores = get_student_grades(student_name)\n    if not scores:  # 如果没有成绩，返回0.0\n        return 0.0\n    return sum(scores) / len(scores)  # 计算特定学生的平均分\n```"}
    """
    print(len(fix_dict_list))

    for op in fix_dict_list:
        op_flag = op["flag"]
        print(op)
        if op_flag == "update":
            start_line = op["start"]
            end_line = op["end"]
            code_fix1 = delete_line(code, start_line, end_line)
            code_to_add = code_extract(op["code_to_add"])
            code_fix2 = add_line(code_fix1, start_line, code_to_add)
            print("更新后的代码为：")
            print(code_fix2)
            code = code_fix2
            # return code_fix2
        elif op_flag == "del":
            start_line = op["start"]
            end_line = op["end"]
            code_fix1 = delete_line(code, start_line, end_line)
            print("更新后的代码为：")
            print(code_fix1)
            code = code_fix1
            # return code_fix1
        elif op_flag == "add":
            start_line = op["start"]
            op["code_to_add"] = code_extract(op["code_to_add"])
            code_to_add = code_extract(op["code_to_add"])
            code_fix1 = add_line(code, start_line, code_to_add)
            print("更新后的代码为：")
            print(code_fix1)
            code = code_fix1
            # return code_fix1
    return code

In [28]:
import regex
import json

# 递归正则表达式
matches = regex.findall(
    r"\{(?:[^{}]|(?R))*\}",
    resp_2.content.replace("’", "'").replace("‘", "'").split("[OUTPUT]")[1],
)
for _ in matches:
    print(_)

print("=================")


def process_construct(process: str):
    """
    接收一个字符串{}，利用正则表达式来转为一个字典，代表一个操作
    """
    pattern = r"flag:(.*?),"
    match = re.search(pattern, process, re.DOTALL)
    if not match:
        return
    flag = match.group(1).strip()

    if flag == "update":
        pattern = r"flag:(.*?),start:(.*?),end:(.*?),code_to_add:(```.*```)}"
        match = re.search(pattern, process, re.DOTALL)
        if match:
            start = match.group(2).strip()
            end = match.group(3).strip()
            code_to_add = match.group(4)
            return {
                "flag": flag,
                "start": int(start),
                "end": int(end),
                "code_to_add": code_to_add,
            }
    elif flag == "add":
        pattern = r"flag:(.*?),start:(.*?),code_to_add:(```.*```)}"
        match = re.search(pattern, process, re.DOTALL)
        if match:
            start = match.group(2).strip()
            code_to_add = match.group(3)
            return {"flag": flag, "start": int(start), "code_to_add": code_to_add}
    elif flag == "del":
        pattern = r"flag:(.*?),start:(.*?),end:(.*?)}"
        match = re.search(pattern, process, re.DOTALL)
        if match:
            start = match.group(2).strip()
            end = match.group(3).strip()
            return {"flag": flag, "start": int(start), "end": int(end)}


operations = []
for _ in matches:
    print(_)
    operations.append(process_construct(_))

operations.sort(key=lambda x: x["start"], reverse=True)
print(operations)

{flag:add,start:45,code_to_add:```python
def calculate_student_average(student_name):
    scores = get_student_grades(student_name)
    if not scores:
        return 0.0
    return sum(scores) / len(scores)
```}
{flag:update,start:5,end:8,code_to_add:```python
students_dict = {}

def add_student(name, age):
    student_id = len(students) + 1
    student_info = {"id": student_id, "name": name, "age": age}
    students.append(student_info)
    students_dict[name] = student_info  # 存储到字典中
    grades.append({"student_id": student_id, "scores": []})
```}
{flag:update,start:11,end:23,code_to_add:```python
def add_grade(student_name, score):
    student_info = students_dict.get(student_name)
    if not student_info:
        print(f"找不到学生 {student_name}")
        return
    
    student_id = student_info["id"]
    for g in grades:
        if g["student_id"] == student_id:
            g["scores"].append(score)
            break
```}
{flag:del,start:55,end:58}
{flag:add,start:45,code_to_add:```p

In [29]:
repair_code = code_process(code, operations)
with open("example.py", "w", encoding="utf-8") as f:
    f.write(repair_code)

4
{'flag': 'del', 'start': 55, 'end': 58}
更新后的代码为：
students = []
grades = []


def add_student(name, age):
    student_id = len(students) + 1
    students.append({"id": student_id, "name": name, "age": age})
    grades.append({"student_id": student_id, "scores": []})


def add_grade(student_name, score):
    for student in students:
        if student["name"] == student_name:
            student_id = student["id"]
            break
    else:
        print(f"找不到学生 {student_name}")
        return

    for g in grades:
        if g["student_id"] == student_id:
            g["scores"].append(score)
            break


def show_all_students():
    for student in students:
        print(f"ID: {student['id']}, 姓名: {student['name']}, 年龄: {student['age']}")


def get_student_grades(student_name):
    for student in students:
        if student["name"] == student_name:
            student_id = student["id"]
            break
    else:
        print(f"找不到学生 {student_name}")
        return []

   